In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# -------------------------
# Load raw CSV
# -------------------------
df = pd.read_csv("/content/imdb_query_plan_features.csv")

# -------------------------
# Select relevant columns
# -------------------------
cols_to_use = [
    'query_index', 'query', 'node_type', 'parallel_aware',
    'startup_cost (ms)', 'total_cost (ms)', 'plan_rows', 'plan_width'
]
df = df[cols_to_use]

# -------------------------
# Preprocess categorical features
# -------------------------
# Encode categorical features (query, node_type, parallel_aware) using LabelEncoder
categorical_cols = ['query', 'node_type', 'parallel_aware']
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = df[col].fillna('missing')  # handle missing values
    df[col] = le.fit_transform(df[col])
    encoders[col] = le  # save encoder for later use if needed

# -------------------------
# Aggregate numeric features by query_index
# -------------------------
numeric_cols = ['startup_cost (ms)', 'total_cost (ms)', 'plan_rows', 'plan_width']
df_agg_numeric = df.groupby('query_index')[numeric_cols].mean().reset_index()

# -------------------------
# Keep one row per query_index for categorical features
# (assuming all rows for the same query_index have same categorical info)
# -------------------------
df_cat = df.groupby('query_index')[categorical_cols].first().reset_index()

# -------------------------
# Merge numeric & categorical
# -------------------------
df_preprocessed = pd.merge(df_agg_numeric, df_cat, on='query_index')

# Optional: round numeric features for readability
df_preprocessed[numeric_cols] = df_preprocessed[numeric_cols].round(2)

# -------------------------
# Save preprocessed file
# -------------------------
preprocessed_file = "/content/query_index_features_aggregated.csv"
df_preprocessed.to_csv(preprocessed_file, index=False)
print(f"✅ Preprocessed data saved to {preprocessed_file}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# -------------------------
# Load CSV
# -------------------------
df = pd.read_csv("/content/imdb_query_plan_features.csv")

# -------------------------
# Features & Target
# -------------------------
features = ['query_index', 'query', 'node_type', 'parallel_aware',
            'startup_cost (ms)', 'plan_rows', 'plan_width']
target = 'total_cost (ms)'

X = df[features]
y_raw = df[target]
y = np.log1p(y_raw)  # log-transform

# -------------------------
# Split numeric & categorical
# -------------------------
numeric_features = ['query_index', 'startup_cost (ms)', 'plan_rows', 'plan_width']
categorical_features = ['query', 'node_type', 'parallel_aware']

# -------------------------
# Preprocessing
# -------------------------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# -------------------------
# Train/Test split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------------
# Custom metrics
# -------------------------
def adjusted_r2(r2, n, k):
    return 1 - (1-r2)*(n-1)/(n-k-1)

def evaluate_model(model, X, y, dataset_name="Model"):
    y_pred = model.predict(X)
    n, k = X.shape
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    adj_r2 = adjusted_r2(r2, n, k)
    return {"Dataset": dataset_name, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "AdjR2": adj_r2}

# -------------------------
# Define models & hyperparameters
# -------------------------
models = {
    "Random Forest": {
        "model": Pipeline([('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))]),
        "params": {
            'regressor__n_estimators': [100, 200],
            'regressor__max_depth': [10, None],
            'regressor__max_features': ['sqrt', 'log2']
        }
    },
    "XGBoost": {
        "model": Pipeline([('preprocessor', preprocessor),
                           ('regressor', XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))]),
        "params": {
            'regressor__n_estimators': [100, 200],
            'regressor__max_depth': [6, 10],
            'regressor__learning_rate': [0.05, 0.1],
            'regressor__subsample': [0.8, 1.0]
        }
    },
    "Gradient Boosting": {
        "model": Pipeline([('preprocessor', preprocessor),
                           ('regressor', GradientBoostingRegressor(random_state=42))]),
        "params": {
            'regressor__n_estimators': [100, 200],
            'regressor__max_depth': [4, 6],
            'regressor__learning_rate': [0.05, 0.1]
        }
    },
    "Linear Regression": {
        "model": Pipeline([('preprocessor', preprocessor),
                           ('regressor', LinearRegression())]),
        "params": {
            'regressor__fit_intercept': [True, False],
            'regressor__positive': [True, False]
        }
    },
    "SVR": {
        "model": Pipeline([('preprocessor', preprocessor),
                           ('regressor', SVR())]),
        "params": {
            'regressor__kernel': ['linear', 'rbf'],
            'regressor__C': [0.1, 1, 10],
            'regressor__gamma': ['scale', 'auto']
        }
    }
}

# -------------------------
# Train & evaluate all models
# -------------------------
results = []
metrics_list = []

for name, m in models.items():
    print(f"\n🔹 Training {name}...")
    grid = GridSearchCV(m["model"], m["params"], cv=3, scoring="r2", n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    print(f"Best params for {name}: {grid.best_params_}")

    # Evaluate Train and Test
    train_metrics = evaluate_model(best_model, X_train, y_train, "Train")
    test_metrics  = evaluate_model(best_model, X_test, y_test, "Test")

    # Append metrics to list
    train_metrics["Model"] = name
    test_metrics["Model"] = name
    metrics_list.append(train_metrics)
    metrics_list.append(test_metrics)

    results.append((name, best_model, test_metrics["R2"]))

# -------------------------
# Save metrics to CSV
# -------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv("/content/model_metrics_train_test.csv", index=False)
print("\n✅ Metrics saved to /content/model_metrics_train_test.csv")

# -------------------------
# Select & save best model (based on Test R²)
# -------------------------
best_model_name, best_model, _ = max(results, key=lambda x: x[2])
print(f"\n🏆 Best Model: {best_model_name}")
joblib.dump(best_model, f"/content/best_{best_model_name.replace(' ','_').lower()}.pkl")


KeyError: "['query', 'node_type', 'parallel_aware'] not in index"